**Análisis de sentimientos de los comentarios de las películas IMDB**

**Problema a resolver:**

Tenemos que predecir el número de comentarios positivos y negativos a partir de un análisis de sentimientos usando diferentes modelos de clasificación.

**Importamos las librerias**

In [ ]:
#Cargar las librerías
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings('ignore')


**Importar el dataset de entrenamiento**

In [ ]:
#importando el dataset de entrenamiento
imdb_data=pd.read_csv('../input/IMDB Dataset.csv')
nRow, nCol = imdb_data.shape
print(f'Hay {nRow} filas y {nCol} columnas')
#Se ajusta el tamaño
print('Se ajusta el tamaño')
imdb_data= imdb_data.head(2000)
nRow, nCol = imdb_data.shape
print(f'Hay {nRow} filas y {nCol} columnas')
imdb_data.head(10)

**Análisis exploratorio de los datos**

In [ ]:
#Resumen del dataset
imdb_data.describe()

**Contador de sentimientos**

In [ ]:
#contador de sentimientos
imdb_data['sentiment'].value_counts()

Podemos apreciar que el dataset está equilibrado.

**Dividiendo el Dataset de entrenamiento**

In [ ]:
#dividiendo el dataset 
#dataset de entrenamiento
train_reviews=imdb_data.review[:1000]
train_sentiments=imdb_data.sentiment[:1000]
#dataset de prueba
test_reviews=imdb_data.review[1000:]
test_sentiments=imdb_data.sentiment[1000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

**Normalización del texto**

In [ ]:
#Tokenización del texto
tokenizer=ToktokTokenizer()
#Poniendo las "Stopwords" en inglés
stopword_list=nltk.corpus.stopwords.words('english')

**Borrando restos de html y texto sobrante**

In [ ]:
#Borrando restos de html
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Borrando corchetes
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

**Borrando caracteres especiales**

In [ ]:
#definimos la función para borrar los caracteres especiales
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Aplicamos la función en la columna "review"
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

**Retocando el texto**

**Borrando las "stopwords"**

In [ ]:
#Retocando el texto
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Aplicar la función en la columna "review"
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [ ]:
#Creando las "stopwords" en inglés
stop=set(stopwords.words('english'))
print(stop)

#borrando las stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Aplicar la función a la columna review
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

**Normalizar los comentarios de entrenamiento**

In [ ]:
#normalizar los comentarios de entrenamiento
norm_train_reviews=imdb_data.review[:1000]
norm_train_reviews[0]
#convertir el dataset string
#norm_train_string=norm_train_reviews.to_string()
#Corrección de ortografía usando Textblob
#norm_train_spelling=TextBlob(norm_train_string)
#norm_train_spelling.correct()
#Tokenizacion usando Textblob
#norm_train_words=norm_train_spelling.words
#norm_train_words

**Normalizar los comentarios de prueba**

In [ ]:
#Normalizar los comentarios de prueba
norm_test_reviews=imdb_data.review[1000:]
norm_test_reviews[1005]
##conovertir el dataframe en string
#norm_test_string=norm_test_reviews.to_string()
#corrección de ortografía usando Textblob
#norm_test_spelling=TextBlob(norm_test_string)
#print(norm_test_spelling.correct())
#Tokenización usando Textblob
#norm_test_words=norm_test_spelling.words
#norm_test_words

**Bags of words modelo **

Se usa para convertir texto a vectores numéricos (Bag of words).

In [ ]:
#Contar "Vectorizer" para el bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#comentarios de entrenamiento transformados
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#comentarios de prueba transformados
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

**Term Frequency-Inverse Document Frequency model (TFIDF)**

 Se usa para convertir documentos de texto en matrices con caracteristicas de TFIDF

In [ ]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#comentarios de entrenamiento transformados
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#comentarios de prueba transformados
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

**Etiquetando el texto de sentimientos**

In [ ]:
#Etiquetando el texto de sentimientos
lb=LabelBinarizer()
#datos de sentimientos transformados
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

**Dividir los datos de los sentimientos**

In [ ]:
#Dividir los datos de los sentimientos
train_sentiments=sentiment_data[:1000]
test_sentiments=sentiment_data[1000:]
print(train_sentiments)
print(test_sentiments)

**Modelando el dataset**

**Nos permiteconstruir un modelo de regresion logistica para las dos bag of words y funciones tfidf.**

In [ ]:
#Entrenando el modelo
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Ajustando el modelo para la Bag Of Words
lr_bow=lr.fit(cv_train_reviews,train_sentiments)
print(lr_bow)
#Ajustando el modelo para las funciones tfidf
lr_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(lr_tfidf)

**Modelo de regresion logistica sobre el test dataset.**

In [ ]:
#Prediciendo el modelo de bag of words
lr_bow_predict=lr.predict(cv_test_reviews)
print(lr_bow_predict)
##Prediciendo el modelo para caracteristicas tfidf
lr_tfidf_predict=lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

**Eficacia del modelo**

In [ ]:
#Puntuación de la eficacia para bag of words
lr_bow_score=accuracy_score(test_sentiments,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Puntuación de la eficacia para caracteristicas tfidf 
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

**Print de la clasificación**

In [ ]:
#Clasificación para bag of words 
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print(lr_bow_report)

#Clasificación para las características tfidf
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print(lr_tfidf_report)

**Matriz de confusión**

In [ ]:
#matriz de confusión de la bag of words
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print(cm_tfidf)

**Descenso gradiente estocástico o máquinas de vectores de soporte(svm) para las bag of words y características tfidf**

In [ ]:
#Entrenando el svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)
#Ajustando el svm para bag of words
svm_bow=svm.fit(cv_train_reviews,train_sentiments)
print(svm_bow)
#Ajustando el svm para características tfidf 
svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)
print(svm_tfidf)

**Modelo del rendimiento sobre la información de prueba**

In [ ]:
#Prediciendo el modelo para la bag of words
svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)
#Prediciendo el modelo para las características tfidf 
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

**Exactitud del modelo**

In [ ]:
#Exactitud de la puntuación para la bag of words
svm_bow_score=accuracy_score(test_sentiments,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)
#Exactitud de la puntuación para las caracteristicas tfidf 
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
print("svm_tfidf_score :",svm_tfidf_score)

**Print de la clasificación**

In [ ]:
#Clasificación para la bag of words
svm_bow_report=classification_report(test_sentiments,svm_bow_predict,target_names=['Positive','Negative'])
print(svm_bow_report)
#Clasificación para las caracteristicas tfidf
svm_tfidf_report=classification_report(test_sentiments,svm_tfidf_predict,target_names=['Positive','Negative'])
print(svm_tfidf_report)

**Grafico de la matriz de confusión**

In [ ]:
#Matriz de confusión para la bag of words
cm_bow=confusion_matrix(test_sentiments,svm_bow_predict,labels=[1,0])
print(cm_bow)
#Matriz de confusión para características tfidf 
cm_tfidf=confusion_matrix(test_sentiments,svm_tfidf_predict,labels=[1,0])
print(cm_tfidf)

**Multinomial Naive Bayes para las bag of words y características tfidf**

In [ ]:
#entrenando el modelo
mnb=MultinomialNB()
#justando el svm para bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#ajustando el svm para las características tfidf 
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

**Modelo de rendimiento en la información de prueba**

In [ ]:
#Prediciendo el modelo para las bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Prediciendo el modelo para las caractristicas tfidf
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

**Exctitud del modelo**

In [ ]:
#Exactitud de la puntuación para las bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Exactitud de la puntuación para las características tfidf
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

**Print de la clasificación**

In [ ]:
#Clasificación para las bag of words 
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Clasificación para las características tfidf 
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

**Grafico de la matriz de confusion**

In [ ]:
#confusion matrix para la bag of words
cm_bow=confusion_matrix(test_sentiments,mnb_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix para las características tfidf 
cm_tfidf=confusion_matrix(test_sentiments,mnb_tfidf_predict,labels=[1,0])
print(cm_tfidf)

**Veamos cuales han sido las palabras positivas y negativas de los comentarios usando WordCloud**

**Word cloud para las palabras de los comentarios positivos**

In [ ]:
#Word cloud para las palabras de los comentarios positivos
plt.figure(figsize=(10,10))
positive_text=norm_train_reviews[1]
WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
positive_words=WC.generate(positive_text)
plt.imshow(positive_words,interpolation='bilinear')
plt.show

**Word cloud para las palabras de los comentarios negativos**

In [ ]:
#Word cloud para las palabras de los comentarios negativos
plt.figure(figsize=(10,10))
negative_text=norm_train_reviews[8]
WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
negative_words=WC.generate(negative_text)
plt.imshow(negative_words,interpolation='bilinear')
plt.show

**Conclusion:**
* We can observed that both logistic regression and multinomial naive bayes model performing well compared to linear support vector  machines.
* Still we can improve the accuracy of the models by preprocessing data and by using lexicon models like Textblob.